In [26]:
import folium
from folium import plugins
from geopy.distance import geodesic
import pandas as pd

In [14]:
df = pd.read_csv('cleaned_data.csv')

In [46]:
# 创建Folium地图，以第一个发射地点的坐标为中心
map_object = folium.Map(location=[df['Latitude'].iloc[0], df['Longitude'].iloc[0]], zoom_start=8)

# 为每个发射地点添加圆形区域和文本标签
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=50,
        color='orange',
        fill=True,
        fill_color='orange',
        popup=row['LaunchSite']
    ).add_to(map_object)

# 创建标记群集
marker_cluster = plugins.MarkerCluster().add_to(map_object)

# 为每个发射地点的成功/失败添加标记
for index, row in df.iterrows():
    result_color = 'green' if row['Outcome'] else 'red'
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.Icon(color=result_color),
        popup=f"{row['LaunchSite']} - {'Outcome' if row['Outcome'] else 'Failed'}"
    ).add_to(marker_cluster)


In [47]:
# 最近海岸线的坐标
CCSFS = (28.5618571,-80.577366)

coastline_CCSFS = (28.563196629377874, -80.56813738999656)
railway_CCSFS = (28.56340440386078, -80.58696293172106)
highway_CCSFS = (28.55957722836178, -80.57776109464285)

dis_coastline_CCSFS = geodesic(CCSFS, coastline_CCSFS).kilometers
dis_railway_CCSFS = geodesic(CCSFS, railway_CCSFS).kilometers
dis_highway_CCSFS = geodesic(CCSFS, highway_CCSFS).kilometers

folium.Marker(
    location=KSC,
    popup='CCSFS'
).add_to(map_object)

folium.Marker(
    location=coastline_KSC,
    popup='CCSFS_Closest Coastline'
).add_to(map_object)

# 在地图上画线
folium.PolyLine(
    locations=[CCSFS, coastline_CCSFS],
    color='blue',
    weight=2.5,
    opacity=1
).add_to(map_object)

folium.PolyLine(
    locations=[CCSFS, railway_CCSFS],
    color='blue',
    weight=2.5,
    opacity=1
).add_to(map_object)

folium.PolyLine(
    locations=[CCSFS, highway_CCSFS],
    color='blue',
    weight=2.5,
    opacity=1
).add_to(map_object)

# 在线上显示距离
distance_popup = f'Distance: {dis_coastline_CCSFS:.2f} km'
folium.Marker(
    location=[(CCSFS[0] + coastline_CCSFS[0]) / 2, (CCSFS[1] + coastline_CCSFS[1]) / 2],  
    popup=distance_popup,
    icon=folium.Icon(color='red')
).add_to(map_object)

distance_popup = f'Distance: {dis_railway_CCSFS:.2f} km'
folium.Marker(
    location=[(CCSFS[0] + railway_CCSFS[0]) / 2, (CCSFS[1] + railway_CCSFS[1]) / 2],  
    popup=distance_popup,
    icon=folium.Icon(color='red')
).add_to(map_object)

distance_popup = f'Distance: {dis_highway_CCSFS:.2f} km'
folium.Marker(
    location=[(CCSFS[0] + highway_CCSFS[0]) / 2, (CCSFS[1] + highway_CCSFS[1]) / 2],  
    popup=distance_popup,
    icon=folium.Icon(color='red')
).add_to(map_object)

In [48]:
# 保存地图为 HTML 文件
map_object.save('launch_sites_map.html')